# Tanpopo1 表面付着物 VGG16+BatchNorm

In [1]:
# Google Colab マウント
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
import os
os.chdir('/content/drive/MyDrive/Tanpopo')

from __future__ import print_function, division

import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data

import matplotlib.pyplot as plt
import glob
import time
import copy
from PIL import Image

plt.ion()

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
#画像サイズがが704x480 #88x60
img_size = 224

#class_names = ['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']

# 標準化
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
batch_size =  "16" #@param[8, 16, 32, 64, 128, 256]
batch_size = int(batch_size)

epochs = "22" #@param[5, 8, 10, 15, 20, 22, 25, 27, 29, 30, 31, 32, 33, 35, 45, 60, 120]
epochs = int(epochs)

### 関数、クラスの定義

In [4]:
import random
import glob
from sklearn.model_selection import train_test_split

def make_filepath_list(folderpath, phase='train'):
    """
    ファイルのパスを格納したリストを返す
    """
    # .DS_Storeが最初に読み込まれる
    test_file_list = []
    train_file_list = []
    valid_file_list = []
    class_names = []

    for index, top_dir in enumerate(sorted(os.listdir(folderpath))):
        file_dir = os.path.join(folderpath, top_dir)
        file_list = glob.glob(file_dir + '/*bmp')

        if top_dir != '.DS_Store':
            class_names.append(top_dir)

            if phase == 'test': # テストデータの場合
                test_file_list += [os.path.join(folderpath, top_dir, file).replace('\\', '/') for file in file_list]
                                                            
            else:
                # 各クラス(フォルダ)ごとに8割を訓練データ、2割を検証データとする
                file_list += [os.path.join(folderpath, top_dir, file).replace('\\', '/') for file in file_list]
                train_file, valid_file = train_test_split(file_list, train_size=0.8, test_size=0.2, random_state=0)
                train_file_list += train_file
                valid_file_list += valid_file

    if phase == 'test':
        return test_file_list, class_names

    return train_file_list, valid_file_list, class_names

In [5]:
class ImageTransform(object):
    """
    画像の前処理
    """
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # データオグメンテーション, 前処理
                transforms.Resize(256), # リサイズ
                transforms.CenterCrop(resize), # 切り取り
                transforms.RandomRotation(45), # ランタムに回転
                transforms.ColorJitter(), # ランダムに明るさ、コントラスト、彩度、色相を変化
                transforms.RandomHorizontalFlip(), # ランダムに左右(水平)反転
                transforms.RandomVerticalFlip(), # ランダムに上下(垂直)反転
                transforms.ToTensor(),
                transforms.Normalize(mean, std), # zcaと交換？
                # ZCA whitening追加する
            ]),
            'valid': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]),
            'test': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

In [6]:
from torchvision.io import read_image

class SurfaceObjectDataset(data.Dataset):
    """
    表面付着物のDatasetクラス
    PyTorchのDatasetクラスを継承
    """
    def __init__(self, file_list, classes, transform=None, phase='train'):
        #super().__init__()
        self.file_list = file_list
        self.transform = transform
        self.classes = classes
        self.phase = phase

        self.img = None
        self.label = None

    def __len__(self):
        """
        画像の枚数を返す
        """
        return len(self.file_list)

    def __getitem__(self, index):
        """
        前処理した画像データのTensor形式のデータとラベルを取得
        """
        # 指定したindexの画像を読み込む
        img_path = self.file_list[index]
        img = Image.open(img_path)

        # 画像ラベルをファイル名から抜き出す
        label = self.file_list[index].split('/')[6][:11]

        # ラベル名を数値に変換
        label = self.classes.index(label)

        # 画像の前処理を実施
        if self.transform is not None:
            img_transformed = self.transform(img, self.phase)
        
        self.img = img_transformed
        self.label = label

        return img_transformed, label

### データ読み込み・前処理

In [7]:
# 訓練、検証データへのファイルパスを格納したリストを取得
train_file_list, valid_file_list, class_names = make_filepath_list('/content/drive/MyDrive/Tanpopo/TrainingData11', 'train')

print('train_file_list: ', train_file_list)
print('class_names: ', class_names)
class_num = len(class_names) # 5

# Datasetの作成
train_dataset = SurfaceObjectDataset(
    file_list = train_file_list, classes = class_names,
    transform = ImageTransform(img_size, mean, std),
    phase = 'train'
)

valid_dataset = SurfaceObjectDataset(
    file_list = valid_file_list, classes = class_names,
    transform = ImageTransform(img_size, mean, std),
    phase = 'valid'
)

# Dataloaderの作成
train_dataloader = data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle=True
)

valid_dataloader = data.DataLoader(
    valid_dataset, batch_size = int(batch_size/2), shuffle=False
)

dataloaders_dict = {
    'train': train_dataloader,
    'valid': valid_dataloader
}

# # イテレータに変換
# batch_iterator = iter(dataloaders_dict['train'])

# inputs, labels = next(batch_iterator)

train_file_list:  ['/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0216S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0055S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SF3A0057S010.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0153S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0254S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0279S011.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SF3A0098S010.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0279S011.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0157S008.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0071S014.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SG3A0024S000.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SM3A0154S009.bmp', '/content/drive/MyDrive/Tanpopo/TrainingData11/1Sputter/SP3A0127S000.bmp', '/cont

### モデルの作成

In [8]:
# モデルをロード
# VGG16学習済みの重みを使用
model_vgg16 = models.vgg16(pretrained=True)
model_vgg16BatchNorm = None # VGG16+Batch Normalization

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [9]:
print(model_vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
def Add_BatchNorm(pretrained_vgg16, class_num):
    if model_vgg16BatchNorm is None:
        # VGG16 に BatchNorm　を挿入 (ReLU層とMax Pool2d層の間)
        pretrained_vgg16.features.insert(4, nn.BatchNorm2d(64))
        pretrained_vgg16.features.insert(10, nn.BatchNorm2d(128))
        pretrained_vgg16.features.insert(18, nn.BatchNorm2d(256))
        pretrained_vgg16.features.insert(26, nn.BatchNorm2d(512))
        pretrained_vgg16.features.insert(34, nn.BatchNorm2d(512))

        # 最後の層
        pretrained_vgg16.classifier[6] = nn.Linear(4096, class_num)

    return pretrained_vgg16


model_vgg16BatchNorm = Add_BatchNorm(model_vgg16, class_num)
model_vgg16BatchNorm.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplac

In [12]:
# まず全パラメータを勾配計算Falseにする
for param in model_vgg16BatchNorm.parameters():
    param.requires_grad = False

params_to_update = []

# Batch Normalization 重み変更
batch_idx = [4, 10, 18, 26, 34]
for idx in batch_idx:
    for param in model_vgg16BatchNorm.features[idx].parameters():
        param.requires_grad = True
        params_to_update.append(param)
    
# 追加したクラス分類用の全結合層を勾配計算ありに変更
for param in model_vgg16BatchNorm.classifier.parameters():
    param.requires_grad = True
    params_to_update.append(param)

optimizer = optim.AdamW(params_to_update, lr=0.001, weight_decay=0.01) # BatchNormを入れることでlrを大きくできる？
# lr(Learning Rate)を大きくすると局所minになりずらい

criterion = nn.CrossEntropyLoss()

In [14]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
#from torchsummary import summary
from torchinfo import summary
model_vgg16BatchNorm.to(device)
print(summary(model=model_vgg16BatchNorm, input_size=(batch_size, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params'], 
        device=device,
        #verbose=2
))
print('VGG16 + BatchNormalization')

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGG                                      [16, 3, 224, 224]         [16, 5]                   --
├─Sequential: 1-1                        [16, 3, 224, 224]         [16, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [16, 3, 224, 224]         [16, 64, 224, 224]        (1,792)
│    └─ReLU: 2-2                         [16, 64, 224, 224]        [16, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [16, 64, 224, 224]        [16, 64, 224, 224]        (36,928)
│    └─ReLU: 2-4                         [16, 64, 224, 224]        [16, 64, 224, 224]        --
│    └─BatchNorm2d: 2-5                  [16, 64, 224, 224]        [16, 64, 224, 224]        128
│    └─MaxPool2d: 2-6                    [16, 64, 224, 224]        [16, 64, 112, 112]        --
│    └─Conv2d: 2-7                       [16, 64, 112, 112]        [16, 128, 112, 112]       (73,856)
│    └─ReLU: 2-8 

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))

        # 各エポックには訓練フェーズと検証フェーズがあります
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに設定します
            else:
                model.eval()   # モードを評価するモデルを設定します

            running_loss = 0.0
            running_corrects = 0

            # データをイレテートします
            for inputs, labels in dataloaders_dict[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # パラメータの勾配をゼロにします
                optimizer.zero_grad()

                # 順伝播
                # 訓練の時だけ、履歴を保持します
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 訓練の時だけ逆伝播＋オプティマイズを行います
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 損失を計算します
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

            print(' {} Loss: {:.4f} Acc: {:.4f} '.format(
                phase, epoch_loss, epoch_acc), end='\t')

            # モデルをディープ・コピーします
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # ベストモデルの重みをロードします
    model.load_state_dict(best_model_wts)
    return model

### 学習

In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_vgg16BatchNorm_weights.pth'
model_vgg16BatchNorm.load_state_dict(torch.load(PATH)) # 学習前: 前回の重みを使う

<All keys matched successfully>

In [ ]:
# 学習
print('VGG16 + Batch Normalization Training :')
print('-'*10)
model_vgg16BatchNorm = model_vgg16BatchNorm.to(device)
model_vgg16BatchNorm = train_model(model_vgg16BatchNorm, criterion, optimizer, num_epochs=epochs)

VGG16 + Batch Normalization Training :
----------
Epoch 0/31
 train Loss: 0.3619 Acc: 0.8923 	 valid Loss: 0.2070 Acc: 0.9500 	
Epoch 1/31
 train Loss: 0.3725 Acc: 0.8923 	 valid Loss: 0.1809 Acc: 0.9500 	
Epoch 2/31
 train Loss: 0.3348 Acc: 0.9051 	 valid Loss: 0.1957 Acc: 0.9500 	
Epoch 3/31
 train Loss: 0.3073 Acc: 0.9205 	 valid Loss: 0.2362 Acc: 0.9200 	
Epoch 4/31
 train Loss: 0.3560 Acc: 0.8974 	 valid Loss: 0.2175 Acc: 0.9400 	
Epoch 5/31
 train Loss: 0.2649 Acc: 0.9103 	 valid Loss: 0.2141 Acc: 0.9300 	
Epoch 6/31
 train Loss: 0.2585 Acc: 0.9231 	 valid Loss: 0.1975 Acc: 0.9300 	
Epoch 7/31
 train Loss: 0.3945 Acc: 0.8897 	 valid Loss: 0.1881 Acc: 0.9300 	
Epoch 8/31
 train Loss: 0.3073 Acc: 0.9179 	 valid Loss: 0.2309 Acc: 0.9200 	
Epoch 9/31
 train Loss: 0.3029 Acc: 0.9051 	 valid Loss: 0.2163 Acc: 0.9400 	
Epoch 10/31
 train Loss: 0.2710 Acc: 0.9256 	 valid Loss: 0.2337 Acc: 0.9300 	
Epoch 11/31
 train Loss: 0.2499 Acc: 0.9282 	 valid Loss: 0.2143 Acc: 0.9200 	
Epoch 12/31


In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_vgg16BatchNorm_weights.pth'
#torch.save(model_vgg16BatchNorm.state_dict(), PATH) # 重みを保存

### テスト

In [ ]:
PATH = '/content/drive/MyDrive/Tanpopo/model_vgg16BatchNorm_weights.pth'
#model_vgg16BatchNorm.load_state_dict(torch.load(PATH)) # 重みをロード

# テストデータ
test_file_list, class_names_test = make_filepath_list('/content/drive/MyDrive/Tanpopo/TestData11', 'test')

print('test_file_list : ', test_file_list)
print('class_names_test : ', class_names_test)

# Datasetの作成
test_dataset = SurfaceObjectDataset(
    file_list = test_file_list, classes = class_names_test,
    transform = ImageTransform(img_size, mean, std),
    phase = 'test'
)

# Dataloaderの作成
test_dataloader = data.DataLoader(
    test_dataset, batch_size = int(batch_size/2), shuffle=False
)

dataloaders_dict['test'] = test_dataloader

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

#正解率
def class_accuracy(label, conf_mat):
    return (conf_mat[label][label] + (np.sum(conf_mat) - (np.sum(conf_mat[:, label])+np.sum(conf_mat[label])-conf_mat[label][label]))) / np.sum(conf_mat)
    
#精度(適合率)
def class_precision(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[label])

#再現率
def class_recall(label, conf_mat):
    return conf_mat[label][label] / np.sum(conf_mat[:, label])

labels_sum = None
predicted_sum = None

with torch.no_grad():
    for data in dataloaders_dict['test']:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = model_vgg16BatchNorm(images)

        #outputs = nn.Softmax(dim=1)(outputs)

        _, predicted = torch.max(outputs, 1)

        if labels_sum is None:
            labels_sum = labels
            predicted_sum = predicted
        else:
            labels_sum = torch.cat([labels_sum, labels], dim=0)
            predicted_sum = torch.cat([predicted_sum, predicted], dim=0)

In [ ]:
#混同行列
labels_sum = labels_sum.cpu()
predicted_sum = predicted_sum.cpu()
conf_mat = None
print('VGG16 Batch Normlization network')
print(class_names_test)

Accuracy = []
Precision = []
Recall = []

conf_mat = confusion_matrix(labels_sum, predicted_sum)
print(conf_mat)
print()
for i in range(class_num):
    Accuracy = np.append(Accuracy, class_accuracy(i, conf_mat)*100)
    Precision = np.append(Precision, class_precision(i, conf_mat)*100)
    Recall = np.append(Recall, class_recall(i, conf_mat)*100)

np.set_printoptions(precision=1)

print('Accuracy : ', Accuracy)
print('Recall : ', Recall)
print('Precision : ', Precision)

VGG16 Batch Normlization network
['1Sputter', '2Fiber', '3Block', '4Bar', '5AGFragment']
[[21  0  0  0  0]
 [ 0 20  0  1  0]
 [ 0  0 21  0  0]
 [ 0  0  0 21  0]
 [ 0  1  0  0 20]]

Accuracy :  [100.   98.1 100.   99.   99. ]
Recall :  [100.   95.2 100.   95.5 100. ]
Precision :  [100.   95.2 100.  100.   95.2]
